In [1]:
import pandas as pd
import numpy as np
import re
from preprocessing import *
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


# Sequence Model Classification with Skip-Gram Embeddings

### Read the data

In [2]:
##############For Training Data#####################
# Read the data from the csv file named 'preprocessedData.csv' arabic data
train = pd.read_csv('../Dataset/train.csv', encoding='utf-8')
# Perform the data preprocessing
train = clean_data(train)
# Unpack the data into text and stance
Train_X = train['text']
# Tokenize the data into words as Skip-Gram model requires a list of words as input
Train_X = [x.split(" ") for x in Train_X]
Train_Y = train['stance']

##############For Testing Data#####################
test = pd.read_csv('../Dataset/dev.csv', encoding='utf-8')
# Perform the data preprocessing
test = clean_data(test)
# Unpack the data into text, and stance
Test_X = test['text']
# Tokenize the data into words as Skip-Gram model requires a list of words as input
Test_X = [x.split(" ") for x in Test_X]
Test_Y = test['stance']


### Use the pre-trained skip-gram model

In [3]:
# Here we want to apply Skip-Gram model to the data
import gensim
from gensim.models import KeyedVectors

# Load the model
sg_model = KeyedVectors.load_word2vec_format('./Word2VecSkipGram300D.bin', binary=True)

### Store the embeddings as features

In [4]:
# Loop over the training data and replace each word with its embedding
# Store the embedding in a different array
max_len = 0
for i in range(len(Train_X)):
    if len(Train_X[i]) > max_len:
        max_len = len(Train_X[i])
counter = 0
Train_X_sg = np.zeros((len(Train_X), max_len, 300))
for i in range(len(Train_X)):
    for j in range(len(Train_X[i])):
        if Train_X[i][j] in sg_model:
            Train_X_sg[i][j] = sg_model[Train_X[i][j]]
        else:
            counter += 1
            print(Train_X[i][j])
print(counter)

الابرة
السيرنجة
الصحة
هلأ
بمؤتمروا
مشكلة
بفايزر
الآمن
لأ
إنوا
الدولة
المارقة
اد
المسؤولية
ألفي
جرعة
لأن
النظافة
وقف
الخطأ
الثلاجة
الصحة
قوجة
جرعة
كورونافاك
بالعاصمة
الخليجية
طلة
اعلامية
ويتسافه
وأخذ
الأجدر
أميركا
يلي
سنة
مؤامرة
عشوا
قصة
بنذكر
الدولة
الضريبة
نج
الإنفوغراف
مؤسسة
أوكسفام
دولة
إسلامية
بكل
المملكة
المتحدة
تأجيل
الجرعة
الثانية
المتاحة
عدد
الجرعة
الثانية
مدة
الجرعة
وحذت
حراموس
الولائيون
لاينفعهم
الأكببببر
يأخذون
الطعوم
إعلان
دولة
وزارة
الصحة
العامة
أثناء
السلالة
الجديدة
وبالمرة
السلالة
السرعة
السلالة
الجديدة
ستأتي
لعضكم
الصحة
خطيرة
وأضاف
ماجوفولي
ضد
الإيدز
فترة
طويلة
ماجوفولي
التنزانيين
مواصلة
الوقت
أعلن
أنه
يكون
الصحة
أنه
يمكن
الجرعة
الأولى
بيونيتك
الجرعة
الثانية
أسترازينيكا
الأولية
بأن
السلالة
الجديدة
المختلفة
نعة
الحصبة
الإصابة
بالحصبة
الإصابة
كوذيب
الإصابة
أنفاسك
وأن
الأجرب
بصائرهم
الجرعة
الأولى
الجرعة
الثانية
يكون
حماية
الثرية
عودة
الحياة
ظل
كأمم
متحدة
القاهرة
شحنة
جرعة
استرازينيكا
وللبد
بالدولة
اللبنانية
الرؤسا
خدمة
حول
هالفئة
عاللقاح
الوقاحة
ونظرة
نظرة
الجرعة
الأولى
لأ

In [5]:
print(Train_X_sg[0])

[[-0.08078194  0.27187496  0.11403476 ... -0.31030625  0.3611038
   0.10022667]
 [ 0.25779235  0.24723987  0.08546001 ... -0.2597796   0.47936234
  -0.08054286]
 [-0.17137918 -0.39501813  0.03944789 ... -0.09690773 -0.24274877
   0.45391282]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


### Reshape the data to apply SMOTE

In [26]:
Train_X_sg_shaped = np.reshape(Train_X_sg, (len(Train_X_sg), -1))

### Apply SMOTE to the training data to balance the classes

In [27]:
# Here we want to apply SMOTE to the data to balance the data against 3 classes
# Count the number of each class
from collections import Counter
from imblearn.over_sampling import SMOTE
print(Counter(Train_Y))
# transform the dataset
oversample = SMOTE()
SMOTE_Train_X_sg, SMOTE_Train_Y = oversample.fit_resample(Train_X_sg_shaped, Train_Y)
print(Counter(SMOTE_Train_Y))


Counter({1: 5538, 0: 1012, -1: 438})
Counter({1: 5538, 0: 5538, -1: 5538})


### Reshape the data back to the original shape

In [28]:
# Reshape the data to be 3D
SMOTE_Train_X_sg = np.reshape(SMOTE_Train_X_sg, (len(SMOTE_Train_X_sg), max_len, 300))


In [29]:
print(SMOTE_Train_X_sg.shape)

(16614, 88, 300)


### Train the model against the unbalanced data

In [6]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
lstm_model = Sequential()
lstm_model.add(LSTM(128, input_shape=(max_len, 300), return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(64))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(lstm_model.summary())
Train_Y = np.array(Train_Y) + 1
# Fit the lstm_model
history = lstm_model.fit(Train_X_sg, Train_Y, epochs=12, batch_size=32, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 88, 128)           219648    
                                                                 
 dropout (Dropout)           (None, 88, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 269,251
Trainable params: 269,251
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/12
219/219 [=========================

In [11]:
# Create random tensor with the same shape as the training data
tensor = np.random.rand(1, max_len, 300)

In [12]:
yhat = lstm_model.predict(tensor)

1/1 [==============================] - 0s 274ms/step


### Predict the classes of the test data

In [7]:
Test_X_sg = np.zeros((len(Test_X), max_len, 300))
for i in range(len(Test_X)):
    for j in range(len(Test_X[i])):
        if Test_X[i][j] in sg_model:
            Test_X_sg[i][j] = sg_model[Test_X[i][j]]
        else:
            counter += 1
            print(Test_X[i][j])

مجزرة
متعمدة
الإيراني
رئيسة
الجمهورية
المنتخبة
للمقاومة
الإيرانية
حول
المتحدة
إعطاء
جرعة
السعودية
الأولوية
الجرعة
الأولى
آمنة
مراجعة
الإنتاج
بالشركة
القابضة
لإنتاج
الإفريقية
السريرية
وستبدأ
السريرية
اللازمة
وقد
تم
تقنية
وزارة
الصحة
الجرعة
الأولى
سيأخذ
الثانية
السعودية
الأميركي
السعودية
أؤكد
الأميركي
آمن
والكفاءة
السعودية
شهدتهما
غاية
الروعة
الحكومة
علي
اللقطة
مرة
ومرة
المالية
او
علي
المختلفة
ككتلة
واحدة
جرعة
بيونتك
استرازانيكا
دفعة
ضد
بالمحافظة
الدولة
ملأت
أتيحت
فرصة
أسبوع
بجدواه
وللأسف
النجاة
لاتهملوه
مباشرة
كتدشين
إطلاق
حملة
وطنية
عالية
الوتيرة
مليون
إسرائيلي
أسبوعين
وهي
أعلى
نسبة
أعلنت
الإمارات
أنها
ضد
وقالت
أظهرت
للوقاية
أجسام
مضادة
بشكل
الجرعة
الثانية
هيئة
الأركان
العامة
قوة
الإستراتيجية
البحرية
الملكية
السعودية
دة
بقالكو
الوسخة
الإندونيسي
جرعة
للوقاية
الأربعاء
الوقت
أطلقت
الحكومة
مليون
أكبر
دولة
عدد
الأميركي
حول
الجرعة
الأولى
وأمريكا
الأسوار
لعبة
عنقودية
الأمن
بن
الجرعة
الأولى
وزارة
الصحة
شركة
ازينيكا
جرعة
عدة
شركة
وقد
مت
مليار
مختلفة
الصحة
العامة
إخفاء
العلامةعبدالله
بن
مطمئنة
ا

In [8]:
print(Test_X_sg)

[[[ 0.26752615  0.08774724  0.01322738 ... -0.32703575 -0.75381464
   -0.15713969]
  [ 0.05406943 -0.28701347  0.25681898 ... -0.05752148  0.34502459
   -0.48687991]
  [ 0.15436892  0.03722382 -0.19472843 ... -0.30660442 -0.32930699
   -0.1309523 ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.07121274 -0.12235447 -0.38020784 ...  0.09074488  0.13624223
    0.11387897]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.05881365 -0.09232171  0.33015332 ...  0.164163

In [9]:
# Print the shape of the data
print(np.shape(Test_X_sg))

(1000, 88, 300)


In [10]:
y_pred = lstm_model.predict(Test_X_sg)

32/32 [==============================] - 2s 39ms/step


In [13]:
print(y_pred[0])

[0.04712033 0.08128405 0.8715957 ]


In [17]:
# Calculate the F1 score for each class
from sklearn.metrics import f1_score
print("F1 score for each class -> ",f1_score(Test_Y+1, y_pred.argmax(axis=1), average=None))
# Calculate the Macro Average F1 score for the whole data
print("Macro Average F1 score -> ",f1_score(Test_Y+1, y_pred.argmax(axis=1), average='macro'))

F1 score for each class ->  [0.         0.         0.89135255]
Macro Average F1 score ->  0.29711751662971175
